In [1]:
# core scientific libraries
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# secondary libraries
import netCDF4 as nc
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, interplevel, CoordPair, vertcross, g_uvmet, interpline, destagger,
                interp2dxy, ll_to_xy, xy, ALL_TIMES, vinterp)
import metpy.calc as mpcalc
from metpy.units import units

import pytz

from xarray.backends.netCDF4_ import NetCDF4DataStore

import copy 

In [72]:
# Set filepath
#Duine 2019 non fire western case study 
# datdir = '/home/sbarc/wrf/wrf401/sundowners/20170311/run-64452560-z0-SMOIS200/'
# filename = 'wrfout_d04_2017-03-10_18:00:00'

# Non-sundowner
datdir = '../input_data/'
filename = 'wrfout_d03_2017-05-22_00:00:00'

# #Cannon Case Study Non-sundowner
# datdir = '/home/sundowners/non-sundowner/2004/'
# filename = 'wrfout_d04_2004-04-29_00:00:00'

# set output filepath
outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/'
fig_outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/figures/'
dat_outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/data/'


# open wrf Dataset object
filepath = datdir + filename
print('Opened File:', filepath, '\n')

# Open NetCDF Dataset object
f = nc.Dataset(filepath,'r')

# List variable names
#print("Variables:")
#print(f.variables.keys(),'\n')

# Show variavles and all details
#print(f.variables.items(),'\n')

# Show dimension names and sizes
#print("Dimensions:")
#print(f.dimensions.items(),'\n')

# Print all file metadata
#print(f)   # or run 'ncdump -h <filename>' in terminal

Opened File: ../input_data/wrfout_d03_2017-05-22_00:00:00 



In [73]:
# specify desired variables to grad from the wrf out file and their units
des_vars = ["temp", "theta", "ua", "va", "wa", "rh"]
var_units = ["K", "K", "m s-1", "m s-1", "m s-1", None]

# specify the z levels to interpolate to [km]
v_levels = np.arange(0, 21, 0.05)

# specify the time index desired
# indx = 3
indx = ALL_TIMES

# make empty list to append interpolated dataarrays too
data = []

for i in range(len(des_vars)):
    
    # grab var with units at time index
    if (var_units[i] != None):
        tmp = getvar(f, des_vars[i], units = var_units[i], timeidx = indx)
    else:
        tmp = getvar(f, des_vars[i], timeidx = indx)
        
    # interpolate to z levels
    tmp_interp = vinterp(wrfin = f, field = tmp, vert_coord = 'ght_msl', interp_levels = v_levels, timeidx = indx)
   
    print("Interpoalted Variable:" + des_vars[i])
    
    # fix projection labeling for saving as a netcdf https://github.com/NCAR/wrf-python/issues/91
    del tmp_interp.attrs['coordinates']

    tmp_interp.attrs['projection'] = str(tmp_interp.attrs['projection'])
    
    # append to list
    data.append(tmp_interp)
    
    
# merge into one dataset    
data = xr.merge(data)

# add terrian height to dataset after correcting the projection labeling
terr = getvar(f, "ter", units = 'm', timeidx = indx)

del terr.attrs['coordinates']

terr.attrs['projection'] = str(terr.attrs['projection'])
 
data['terr']  = terr  
    
# pull lat and lon vallues and replace current corrdinates
lats = data.XLAT.values[:,0]
lons = data.XLONG.values[0,:]

data['south_north'] = lats
data['west_east'] = lons

# remane dimensions with ugly names
data = data.rename({'south_north':'lat', 'west_east':'lon', 'XTIME':'time', 'XLAT':'lat_grid', 
                    'XLONG':'lon_grid', 'interp_level':'z'})

# change vertical coordinates to meters
data['z'] = data['z']*1000

# fix the time coordinates
data['time'] = data['Time']
data = data.drop("Time")

Interpoalted Variable:temp
Interpoalted Variable:theta
Interpoalted Variable:ua
Interpoalted Variable:va
Interpoalted Variable:wa
Interpoalted Variable:rh


In [74]:
data.to_netcdf(path = dat_outdir + filename)

print('Saved Files As: \n', dat_outdir + filename)

Saved Files As: 
 /home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/data/wrfout_d03_2017-05-22_00:00:00
